In [ ]:
# Import libraries. You may or may not use all of these. - do not edit
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

  Preparing metadata (setup.py) ... done
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [119]:
# get data files - do not edit
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-02-09 21:43:28--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.3’

train-data.tsv.3    100%[===================>] 349.84K  1.74MB/s    in 0.2s    

2023-02-09 21:43:29 (1.74 MB/s) - ‘train-data.tsv.3’ saved [358233/358233]

--2023-02-09 21:43:29--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.3’

valid-data.tsv.3    100%[========

In [120]:
# Read .tsv data and load into pandas dataframe
df_train = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'text'])
df_test = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'text'])

In [121]:
# Change categorical 'ham' or 'spam' data into numeric data
df_train['label'] = pd.Categorical(df_train['label']).codes
df_test['label'] = pd.Categorical(df_test['label']).codes

In [122]:
# Create numpy arrays for training and testing labels
train_labels = df_train['label'].values
test_labels = df_test['label'].values

In [123]:
# Create numpy datasets from training and tesitng dataframes
ds_train = tf.data.Dataset.from_tensor_slices((df_train['text'].values, df_train['label'].values))
ds_test = tf.data.Dataset.from_tensor_slices((df_test['text'].values, df_test['label'].values))
ds_test.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int8, name=None))

In [124]:
# Shuffle data for training and create batches of (text, label) pairs
BUFFER_SIZE = 128
BATCH_SIZE = 32
ds_train = ds_train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [125]:
# Create text encoder
VOCAB_SIZE = 2000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(ds_train.map(lambda text, label: text))

In [126]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'to', 'i', 'you', 'a', 'the', 'u', 'and', 'in', 'is',
       'me', 'my', 'for', 'your', 'of', 'it', 'call', 'have', 'on'],
      dtype='<U19')

In [127]:
# Create model
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [128]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [129]:
# Train model
history = model.fit(ds_train, epochs=10,
                    validation_data=ds_test,
                    validation_steps=30)

Epoch 1/10
131/131 [==============================] - 13s 46ms/step - loss: 0.6337 - accuracy: 0.8660 - val_loss: 0.5217 - val_accuracy: 0.8604
Epoch 2/10
131/131 [==============================] - 3s 26ms/step - loss: 0.4005 - accuracy: 0.8660 - val_loss: 0.2842 - val_accuracy: 0.8604
Epoch 3/10
131/131 [==============================] - 3s 21ms/step - loss: 0.1767 - accuracy: 0.9229 - val_loss: 0.1276 - val_accuracy: 0.9667
Epoch 4/10
131/131 [==============================] - 4s 27ms/step - loss: 0.0889 - accuracy: 0.9773 - val_loss: 0.0911 - val_accuracy: 0.9719
Epoch 5/10
131/131 [==============================] - 3s 20ms/step - loss: 0.0619 - accuracy: 0.9835 - val_loss: 0.0720 - val_accuracy: 0.9760
Epoch 6/10
131/131 [==============================] - 3s 26ms/step - loss: 0.0469 - accuracy: 0.9876 - val_loss: 0.0602 - val_accuracy: 0.9823
Epoch 7/10
131/131 [==============================] - 3s 19ms/step - loss: 0.0362 - accuracy: 0.9914 - val_loss: 0.0564 - val_accuracy: 0.985

In [130]:
loss, accuracy = model.evaluate(ds_test)

44/44 [==============================] - 0s 6ms/step - loss: 0.0552 - accuracy: 0.9849


In [132]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  prediction = model.predict([pred_text])[0][0]
  return [prediction, "ham" if prediction < 0.5 else "spam"]

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 3s 3s/step
[1.1235372, 'spam']


In [133]:
# Run this cell to test your function and model. Do not modify contents. - do not edit
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 29ms/step
You passed the challenge. Great job!
